In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:22pt; line-height:40px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# [RAG 절차]

1. 문서를 읽는다
    %pip install --upgrade --quiet docx2txt
2. 문서를 쪼갠다
    %pip install -qU langchain-text-splitters
3. 쪼갠 문서를 임베딩하여 vector database에 넣음(local에 저장) cf. 클라우드에 저장
    %pip install -q langchain-chroma
4. 질문을 이용해 유사도 검색
5. 유사도 검색한 문서를 LLM에 질문과 함께 전달하여 답변 얻음(렝체인 사용 가능)
    %pip install -q langchain
    (https://smith.langchain.com에서 key생성 .env에 LANGCHAIN_API_KEY로 추가)

# 0. 패키지 설치

In [ ]:
# 문서읽어오기
%pip install --upgrade --quiet docx2txt

In [2]:
# 텍스트를 chunk로 나누는 기능만 있는 경량 모듈
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [3]:
# 벡터DB(로컬DB) 어제의 chromadb가 아님
%pip install -q langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [4]:
# langchain 사용
%pip install -q langchain

Note: you may need to restart the kernel to use updated packages.


# 1. 문서읽기(X)

In [5]:
%%time
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader("./data/소득세법(법률)(제21065호)(20260102).docx")
document = loader.load()

CPU times: total: 4.86 s
Wall time: 5.1 s


In [7]:
len(document)

1

# 2. 문서를 쪼개면서 읽기(O)
- https://docs.langchain.com/oss/python/integrations/splitters

## 2.1 1500토큰씩 쪼개서 읽어오기

In [8]:
import time
start = time.time()
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import TokenTextSplitter
loader = Docx2txtLoader('./data/소득세법(법률)(제21065호)(20260102).docx')
# gpt-4, gpt-4o, gpt-4 turbo, gpt4o-mini, embedding모델들은 다 같은 방식으로 토큰 추출
text_splitter = TokenTextSplitter(
    encoding_name="cl100k_base", #토큰을 세는 방식 이름
    chunk_size=1500,             # chunk 당 토큰 수 기준
    chunk_overlap=200
    # separators = ["\n", "\n\n"]파라미터가 없음
)
documents = loader.load_and_split(text_splitter=text_splitter)
runtime = time.time() - start
print("문서를 쪼개면서 읽는 시간 : ", runtime)

문서를 쪼개면서 읽는 시간 :  6.768277406692505


In [10]:
len(documents) # chunk 수

180

In [17]:
# chunk 글자수
# documents[0].page_content
print([len(document.page_content) for document in documents])

[1699, 1656, 1641, 1650, 1738, 1442, 1287, 1535, 1325, 1619, 1596, 1588, 1566, 1639, 1622, 1559, 1612, 1638, 1573, 1465, 1436, 1609, 1456, 1497, 1635, 1606, 1533, 1649, 1662, 1595, 1603, 1678, 1595, 1637, 1601, 1539, 1561, 1594, 1693, 1708, 1657, 1627, 1636, 1659, 1667, 1595, 1491, 1485, 1645, 1709, 1629, 1617, 1495, 1626, 1612, 1620, 1609, 1576, 1636, 1602, 1556, 1563, 1600, 1616, 1643, 1691, 1635, 1685, 1621, 1631, 1609, 1605, 1603, 1604, 1698, 1686, 1702, 1612, 1539, 1558, 1651, 2060, 1562, 1606, 1557, 1648, 1594, 1615, 1766, 1651, 1690, 1576, 1536, 1553, 1638, 1685, 1693, 1694, 1664, 1529, 1627, 1703, 1675, 1546, 1585, 1687, 1679, 1714, 1603, 1655, 1648, 1495, 1531, 1562, 1594, 1646, 1543, 1449, 1593, 1559, 1521, 1473, 1519, 1545, 1668, 1700, 1692, 1655, 1648, 1741, 1670, 1628, 1639, 1623, 1638, 1642, 1666, 1658, 1594, 1591, 1561, 1641, 1498, 1610, 1567, 1613, 1636, 1619, 1531, 1496, 1702, 1598, 1579, 1627, 1559, 1585, 1665, 1565, 1616, 1564, 1612, 1535, 1512, 1557, 1576, 1628, 165

In [22]:
# chunk 글자수 최대값, 최소값
print(max([len(document.page_content) for document in documents]))
print(min([len(document.page_content) for document in documents]))

2060
955


## 2.2 1500글자 쪼개서 읽어오기

# 3. 쪼갠 문서를 임베딩 -> 벡터 베이터베이스 저장
- 임베딩 모델 : text-embedding-3-large (기본모델 :text-embedding-ada-002)
- 벡터데이터베이스(벡터 store) : chroma

In [30]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
load_dotenv()
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [31]:
# embed_query() 한 문자열을 임베딩 벡터로 전환한 숫자 list를 return
len(embedding.embed_query("소득세법은 어쩌구"))

3072

In [34]:
embedding_vectors = embedding.embed_documents( # 여러 문자열을 임베딩 벡터로
    [
        documents[0].page_content,
        documents[1].page_content
    ]
)

In [41]:
print(len(embedding_vectors), len(embedding_vectors[0]), len(embedding_vectors[1]))
print(embedding_vectors[0][:10])

2 3072 3072
[0.014169108122587204, -0.022027326747775078, 0.009101607836782932, 0.009438703767955303, 0.030725523829460144, 0.0165895726531744, -0.026680365204811096, 0.011428126133978367, -0.014898562803864479, 0.01189232524484396]


In [44]:
%%time
from langchain_chroma import Chroma
# 데이터 처음 저장할 때
# database = Chroma.from_documents(
#     documents=documents, # chunk
#     embedding=embedding, # 임베딩 객체
#     collection_name="tax-collection", # 생략시 이름 랜덤
#     persist_directory="./chroma"      # 생략시 로컬DB에 저장 안 됨. 프로그램 종료시 DB날라감
# )
# 이미 저장된 vector DB(store)를 사용할 때
database = Chroma(
    embedding_function=embedding,
    collection_name="tax-collection",
    persist_directory="./chroma"
)

CPU times: total: 15.6 ms
Wall time: 22.9 ms


In [53]:
results = database._collection.get(include=['embeddings', 'documents', 'metadatas'])
print("데이터 수 :", len(results['ids']))
print("문서 임베딩 차원 수 :", len(results['embeddings'][0]))
print("1째 임베딩 샘플 :", results['embeddings'][1])
print("1번째 원본 :", results['documents'][1][:50])
print("1번째 metadata :", results['metadatas'][1])

데이터 수 : 180
문서 임베딩 차원 수 : 3072
1째 임베딩 샘플 : [ 0.01991478 -0.01470464 -0.00057961 ...  0.0058937  -0.03365059
 -0.00657188]
1번째 원본 : . 구성원 간 이익의 분배비율이 정하여져 있지 아니하나 사실상 구성원별로 이익이 분배되는 
1번째 metadata : {'source': './data/소득세법(법률)(제21065호)(20260102).docx'}


# 4. vector DB에 질문과 유사도 검색(답변 생성을 위한 retrieval)

In [55]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query=query,
                                           k=2) # 기본 k값은 4

In [57]:
#retrieved_docs

In [61]:
# print("\n\n---\n\n".join([doc.page_content for doc in retrieved_docs]))
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

# 5. 유사도검색으로 가져온 문서를 질문과 같이 LLM에 전달하여 답변 생성-1

In [63]:
from langchain_openai import ChatOpenAI
load_dotenv()
llm = ChatOpenAI(model="gpt-4.1-nano")

In [64]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세 전문가입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요
[context]는 다음과 같아요
{retrieved_doc}
질문 : {query}"""

In [65]:
ai_message = llm.invoke(prompt)

In [67]:
ai_message.usage_metadata

{'input_tokens': 2132,
 'output_tokens': 709,
 'total_tokens': 2841,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [69]:
print(ai_message.content)

연봉 5천만원인 직장인의 소득세 산정을 위해서는 먼저 근로소득금액을 기준으로 세액을 계산해야 합니다. 참고하신 [context]에 명시된 세율 및 과세표준을 감안하여 설명드리겠습니다.

1. **근로소득금액 계산**  
- 연봉이 50,000,000원일 경우, 근로소득공제와 표준공제 등을 차감하여 과세표준을 산출합니다.

2. **근로소득공제**  
- 2023년 기준 근로소득공제는 다음과 같습니다. (일반근로자 기준)  
  - 1,200만원 이하: 80%  
  - 1,200만원 초과 ~ 4,600만원 이하: 1,200만원 + (연봉 - 1,200만원) × 25%  
  - 4,600만원 초과: 1,700만원 (최대공제액)  
- 연봉 5천만원인 경우,  
  - 공제액 = 1,200만원 + (50,000,000 - 12,000,000) × 25%  
  - = 1,200만원 + 38,000,000 × 25% = 1,200만원 + 9,500,000 = 2,700만원

3. **과세표준 산출**  
- 표준공제(가족공제 등 별도 공제 포함): 예를 들어, 기본공제 150만원 적용  
- 과세표준 = 연봉 - 근로소득공제(2,700만원) - 기본공제(150만원)  
- = 50,000,000원 - 27,000,000원 - 1,500,000원 = 약 21,700,000원

4. **세율 적용**  
- 과세표준 1,200만원 이하 구간: 6%  
- 1,200만원 초과 4,600만원 이하: 15% (누진공제 적용)  
- 4,600만원 초과: 24% (하지만 이번 과세표준은 2,170만원으로 15% 구간에 해당)  

구체적으로 계산하면:

- 1,200만원까지: 1,200만원 × 6% = 72만원  
- 나머지 9,500,000원(21,700,000 - 12,000,000): 9,500,000 × 15% = 1,425,000원

총 세액 = 72만원 + 14.25만원 = 약 **86.25만원**

이 외에 국민연금, 건강보험료 등 4대 보험료가 별도 공제되어야

# 5. 유사도검색으로 가져온 문서를 질문과 같이 LLM에 전달하여 답변 생성-2

In [82]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")
promptTemplate = ChatPromptTemplate([
    ("system", "당신은 최고의 한국 소득세 전문가입니다"),
    ("human", f"""다음 문맥을 참고하여 질문에 답변하세요.
    답을 모르면 모른다고 말하세요.
    최대 3문장으로 간결하게 답변하세요.
    질문 : {{question}}
    문맥 : {{context}}
    답변 : """)
])
promptTemplate

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 최고의 한국 소득세 전문가입니다'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='다음 문맥을 참고하여 질문에 답변하세요.\n    답을 모르면 모른다고 말하세요.\n    최대 3문장으로 간결하게 답변하세요.\n    질문 : {question}\n    문맥 : {context}\n    답변 : '), additional_kwargs={})])

In [83]:
prompt = promptTemplate.invoke({
                            'context':retrieved_doc, # retrieved_docs보다 추천
                            'question':query
        })

In [84]:
llm.invoke(prompt)

AIMessage(content='연봉 5천만원인 직장인의 소득세는 개인의 공제, 세액공제 등에 따라 다르므로 정확한 계산이 필요합니다. 일반적으로 근로소득세율표와 기본공제, 보험료공제 등을 고려하면 약 700만원~1,000만원 정도의 세금이 부과될 수 있습니다. 정확한 세액은 구체적인 공제 항목과 소득 구성에 따라 달라지니, 세무 전문가와 상담하는 것을 권장드립니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 110, 'prompt_tokens': 2155, 'total_tokens': 2265, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7f8eb7d1f9', 'id': 'chatcmpl-Cvb7LH9aYjYXLjfq07OM5FDbpxlcC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b9bad-ba1b-7833-8a26-ee9472f3d41f-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 2155, 'output_tokens': 110, 'total_tokens': 2265, 'input_token_details': {'audio': 0, 'cach

In [86]:
# 위의 예제를 한번에
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
output_parser.invoke(llm.invoke(promptTemplate.invoke({'context':retrieved_doc,
                                                       'question':query})))

'연봉 5천만원인 직장인의 소득세는 정확한 계산이 필요하지만, 대략적으로 근로소득세율 구간에 따라 10%에서 20% 수준입니다. 구체적인 세액은 각 공제 및 세율 적용 후 결정됩니다. 따라서, 상세 계산을 원하시면 추가 정보가 필요합니다.'

# 6. langchain으로 답변 생성

In [87]:
# 위의 예제를 langchain으로 답변생성
rag_chain = promptTemplate | llm | output_parser
rag_chain.invoke({'context':retrieved_doc,'question':query })

'연봉 5천만원인 직장인의 소득세는 구체적인 공제액과 세율에 따라 달라지며, 대략적인 소득세는 700만원 내외일 수 있습니다. 하지만 정확한 세액은 근로소득공제, 인적공제, 특별공제 등 각종 공제 후 계산되어야 합니다. 상세한 계산을 위해서는 공제 내역과 세법 적용이 필요합니다.'

## langchain 전달
    smith.langchain.com에서 key생성 후 .env에 LANGCHAIN_API_KEY 추가

In [ ]:
# 1. LLM과 임베딩 초기화
llm = ChatOpenAI(model = "gpt-4.1-mini")
embedding = OpenAIEmbeddings(model="text-embedding-3-large")
# 2. vector store load
vectorstore = Chroma(
    embedding_function=embedding,
    collection_name="tax-collection",
    persist_directory="./chroma"
)